# Extracting NDVI data from polygon query June 2022
This code is for downloading Sentinel data using a polygon for a location query

In [1]:
%matplotlib inline

import datacube
import geopandas as gpd
from datacube.utils import geometry
from datacube.utils.cog import write_cog

import sys
sys.path.insert(1, '../Tools/')
from dea_tools.datahandling import load_ard
from dea_tools.plotting import map_shapefile

In [2]:
# Temporary solution to account for Collection 3 data being in a different
# database on the NCI
try:
    dc = datacube.Datacube(app='Analyse_multiple_polygons', env='c3-samples')
except:
    dc = datacube.Datacube(app='Analyse_multiple_polygons')

In [3]:
# Read in area of interest vector
vector_file = './vectors/ningaloo_processing_aoi.shp'
attribute_col = 'aoi'
output_crs = 'EPSG:3577'
gdf = gpd.read_file(vector_file)
geom = geometry.Geometry(gdf.iloc[0].geometry.__geo_interface__, 
                          crs=geometry.CRS(output_crs))

# To plot and visualise
map_shapefile(gdf, attribute=attribute_col)

/home/jovyan/Bart/../Tools/dea_tools/plotting.py:418: FutureWarning: The `map_shapefile` function is deprecated, and will be removed from future versions of `dea-tools`. Please use Geopanda's built-in `.explore` functionality instead.
  warnings.warn("The `map_shapefile` function is deprecated, and will "


Label(value='')

Map(center=[-21.960797363881863, 113.93438124230774], controls=(ZoomControl(options=['position', 'zoom_in_text…

In [4]:
# set up parameters
time_range = ('2019-11-04', '2019-11-04')
products = ['s2b_ard_granule']
measurements = ['nbart_blue', 'nbart_green', 'nbart_red', 'nbart_nir_1']
resolution = (-10, 10)
align = (0, 0)

In [5]:
query = {'time': time_range,
         'geopolygon': geom,
         'measurements': measurements,
         'resolution': resolution,
         'output_crs': output_crs,
         'align': align,
         }

#query

In [6]:
# # Load landsat
ds = load_ard(dc=dc, 
               products=products,
               mask_pixel_quality=False,            
               group_by='solar_day',
               **query)

Finding datasets
    s2b_ard_granule
Loading 1 time steps


CPLReleaseMutex: Error = 1 (Operation not permitted)


In [7]:
# Get time and site variable for naming
time = ds.groupby('time')
site = gdf.loc[0 ,attribute_col]
# Create index if required - NOTE if you don't need it, don't create it to make file smaller
ds['ndvi'] = (ds.nbart_nir_1 - ds.nbart_red)/(ds.nbart_nir_1 + ds.nbart_red)
# print(ds)

In [8]:
# Write GeoTIFF

# for single band (e.g. NDVI) use this
# ndvi_name = f'{site}_{str(time)[57:67]}_ndvi.tif'
# ndvi_out = ds.ndvi.isel(time=0)
# write_cog(geo_im=ndvi_out,
#           fname=tname,
#           overwrite=True)

# for multiband use this
multi_name = f'{site}_{str(time)[57:67]}_multiband.tif'
multi_out = ds.isel(time=0).to_array()
write_cog(geo_im=multi_out,
          fname=multi_name,
          overwrite=True)

PosixPath('ningaloo_2019-11-04_multiband.tif')

Our `results` dictionary will contain `xarray` objects labelled by the unique `attribute column` values we specified in the `Analysis parameters` section:

All output needs to be zipped to archive to enable a single download. From the launcher page start up a Terminal, navigate to what makes sense for you work and use a similar tar command as per below:

In [8]:
## zip everything up prior to download
# start up CLI and do an ls command to make sure processing folder exists (in this example it is SWWMP)
# run tar -czvf NAME.tar.gz SWWMP to zip up work - when unzipping can extract to same folder name